### Creacion de Tablas

Abrir coneccion a SQL

In [1]:
import pyodbc

# Conexión a SQL Server
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=ANAGABRIELAD661;"  
    "DATABASE=InsightHub;"     
    "UID=grupo2;"              
    "PWD=henry;"               
)

cursor = conn.cursor()


TABLA PRODUCTOS

In [3]:
# Definición de tablas
create_table_query = """
CREATE TABLE Productos (
    Brand INT PRIMARY KEY, 
    Description NVARCHAR(255) NOT NULL, 
    Classification TINYINT NOT NULL CHECK (Classification IN (1, 2)), -- Add a comma here
    Size_ml FLOAT, 
    Pack_Count INT
);
"""

# Ejecutar la creación de tablas
cursor.execute(create_table_query)
conn.commit()
print("Tabla creada exitosamente.")


Tabla creada exitosamente.


TABLA VENDEDORES

In [8]:
# SQL query to create the Vendedores table
create_vendors_table_query = """
CREATE TABLE Vendedores (
    VendorNumber INT PRIMARY KEY, -- Vendor number as primary key
    VendorName NVARCHAR(255) NOT NULL -- Name of the vendor
);
"""

# Execute the query to create the table
cursor.execute(create_vendors_table_query)
conn.commit()
print("Tabla 'Vendedores' creada exitosamente.")


Tabla 'Vendedores' creada exitosamente.


hay duplicados en la tabla vendors (solo en vendornumber)

In [11]:
duplicates = vendors_data[vendors_data.duplicated(subset=['VendorNumber'], keep=False)]
print("Duplicate entries in the CSV:\n", duplicates)


Duplicate entries in the CSV:
      VendorNumber                  VendorName
48           2000  southern wine & spirits ne
66           1587         vineyard brands inc
126          2000  southern glazers w&s of ne
131          1587         vineyard brands llc


elijo quedarme con 48 y 66

In [12]:
# Identify and keep specific rows
duplicates_to_keep = [48, 66]  # Indices of rows to keep
vendors_data = vendors_data.loc[duplicates_to_keep]

# Reset index after filtering
vendors_data.reset_index(drop=True, inplace=True)

print("Cleaned data to insert:\n", vendors_data)


Cleaned data to insert:
    VendorNumber                  VendorName
0          2000  southern wine & spirits ne
1          1587         vineyard brands inc


In [14]:
for index, row in vendors_data.iterrows():
    try:
        cursor.execute("""
            INSERT INTO Vendedores (VendorNumber, VendorName)
            VALUES (?, ?)
        """, 
        row['VendorNumber'], 
        row['VendorName'])
    except pyodbc.IntegrityError as e:
        # Handle duplicate primary key error
        print(f"Duplicate entry skipped: VendorNumber {row['VendorNumber']}")



Duplicate entry skipped: VendorNumber 2000
Duplicate entry skipped: VendorNumber 1587


In [15]:
# Identify and keep specific VendorNumbers
vendors_to_keep = [2000, 1587]  # VendorNumber values to keep (48 and 66)

# Filter DataFrame to keep only the specified VendorNumbers
vendors_data_cleaned = vendors_data[vendors_data['VendorNumber'].isin(vendors_to_keep)]

# Reset the index after filtering
vendors_data_cleaned.reset_index(drop=True, inplace=True)

print("Cleaned data to insert:\n", vendors_data_cleaned)

# Insert the cleaned data into the database
for index, row in vendors_data_cleaned.iterrows():
    try:
        cursor.execute("""
            INSERT INTO Vendedores (VendorNumber, VendorName)
            VALUES (?, ?)
        """, 
        row['VendorNumber'], 
        row['VendorName'])
    except pyodbc.IntegrityError:
        print(f"Duplicate entry skipped: VendorNumber {row['VendorNumber']}")

print("Datos insertados exitosamente en la tabla 'Vendedores'.")


Cleaned data to insert:
    VendorNumber                  VendorName
0          2000  southern wine & spirits ne
1          1587         vineyard brands inc
Duplicate entry skipped: VendorNumber 2000
Duplicate entry skipped: VendorNumber 1587
Datos insertados exitosamente en la tabla 'Vendedores'.


Inserto products.csv en la tabla de PRODUCTOS

In [ ]:
# Cargar el archivo CSV
csv_file_path = 'products.csv'
data = pd.read_csv(csv_file_path)


# Insert the data into the table
for index, row in data.iterrows():
    cursor.execute("""
        INSERT INTO Productos (Brand, Description, Classification, Size_ml, Pack_Count)
        VALUES (?, ?, ?, ?, ?)
    """,
    row['Brand'], 
    row['Description'], 
    row['Classification'], 
    row['size_ml'],  # Match case with CSV
    row['pack_count'])  # Match case with CSV

# Confirmar los cambios
conn.commit()

print("Datos insertados exitosamente.")



Datos insertados exitosamente.


TABLA DE SALES

In [16]:
# SQL query to create the Sales table
create_sales_table_query = """
CREATE TABLE Sales (
    SalesID INT PRIMARY KEY, 
    InventoryId NVARCHAR(50), 
    Store INT, 
    Brand INT, 
    Description NVARCHAR(255), 
    Size NVARCHAR(50), 
    SalesQuantity INT, 
    SalesDollars FLOAT, 
    SalesPrice FLOAT, 
    SalesDate DATE, 
    size_ml INT, 
    Classification TINYINT, 
    ExciseTax FLOAT, 
    VendorNumber INT, 
    VendorName NVARCHAR(255), 
    CalculatedSalesDollars FLOAT
);
"""

# Execute the query to create the table
cursor.execute(create_sales_table_query)
conn.commit()

print("Tabla 'Sales' creada exitosamente.")


Tabla 'Sales' creada exitosamente.


SalesID era una clave artificial autoincremental, sin embargo hubo errores, al parecer se duplicaron.. por lo que se decide sacar el id y luego agregarlo artificialmente una vez que la tabla esta cargada

In [ ]:
# Crear la tabla Ventas con los tipos de datos especificados sin salesID
create_sales_table_query = """
CREATE TABLE Ventas (
    InventoryId NVARCHAR(50), 
    Store INT, 
    Brand INT, 
    Description NVARCHAR(255), 
    SalesQuantity INT, 
    SalesDollars FLOAT, 
    SalesPrice FLOAT, 
    SalesDate DATE, 
    ExciseTax FLOAT, 
    VendorNumber INT, 
    size_ml FLOAT, 
    pack_count FLOAT
);
"""
cursor.execute(create_sales_table_query)
conn.commit()
print("Tabla 'Ventas' creada exitosamente.")

# Cargar el archivo CSV limpio
csv_file_path = 'cleaned_sales.csv'  
sales_data = pd.read_csv(csv_file_path)

# Insertar los datos en la tabla Ventas
for index, row in sales_data.iterrows():
    try:
        cursor.execute("""
            INSERT INTO Ventas (
                InventoryId, Store, Brand, Description, 
                SalesQuantity, SalesDollars, SalesPrice, SalesDate, 
                ExciseTax, VendorNumber, size_ml, pack_count
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, 
        row['InventoryId'], 
        row['Store'], 
        row['Brand'], 
        row['Description'], 
        row['SalesQuantity'], 
        row['SalesDollars'], 
        row['SalesPrice'], 
        row['SalesDate'], 
        row['ExciseTax'], 
        row['VendorNumber'], 
        row['size_ml'], 
        row['pack_count']
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")

# Confirmar la transacción
conn.commit()
print("Datos insertados exitosamente.")




Tabla 'Ventas' creada exitosamente.
Datos insertados exitosamente.


In [ ]:

# Step 1: Create the new table with specified columns
cursor.execute("""
CREATE TABLE Ventas_New (
    InventoryId NVARCHAR(50), 
    Store INT, 
    Brand INT, 
    Description NVARCHAR(255), 
    SalesQuantity INT, 
    SalesDollars FLOAT, 
    SalesPrice FLOAT, 
    SalesDate DATE, 
    ExciseTax FLOAT, 
    VendorNumber INT, 
    size_ml FLOAT, 
    pack_count FLOAT
);
""")
conn.commit()
print("Created 'Ventas_New' table successfully.")

# Step 2: Copy data from the old table
cursor.execute("""
INSERT INTO Ventas_New (
    InventoryId, Store, Brand, Description, 
    SalesQuantity, SalesDollars, SalesPrice, SalesDate, 
    ExciseTax, VendorNumber, size_ml, pack_count
)
SELECT 
    InventoryId, Store, Brand, Description, 
    SalesQuantity, SalesDollars, SalesPrice, SalesDate, 
    ExciseTax, VendorNumber, size_ml, pack_count
FROM Ventas;
""")
conn.commit()
print("Copied data to 'Ventas_New' table successfully.")

# Step 3: Drop the old table
cursor.execute("DROP TABLE Ventas;")
conn.commit()
print("Dropped old 'Ventas' table successfully.")

# Step 4: Rename the new table
cursor.execute("EXEC sp_rename 'Ventas_New', 'Ventas';")
conn.commit()
print("Renamed 'Ventas_New' to 'Ventas'.")

# Step 5: Add auto-incrementing primary key
cursor.execute("""
ALTER TABLE Ventas
ADD SalesID INT IDENTITY(1,1) PRIMARY KEY;
""")
conn.commit()
print("Added auto-incrementing primary key 'SalesID' to 'Ventas'.")


Created 'Ventas_New' table successfully.
Copied data to 'Ventas_New' table successfully.
Dropped old 'Ventas' table successfully.
Renamed 'Ventas_New' to 'Ventas'.
Added auto-incrementing primary key 'SalesID' to 'Ventas'.


In [13]:
# Add foreign key for Brand
cursor.execute("""
ALTER TABLE Ventas
ADD CONSTRAINT FK_Ventas_Productos
FOREIGN KEY (Brand) REFERENCES Productos(Brand);
""")
conn.commit()
print("Added foreign key constraint FK_Ventas_Productos.")

# Add foreign key for VendorNumber
cursor.execute("""
ALTER TABLE Ventas
ADD CONSTRAINT FK_Ventas_Vendedores
FOREIGN KEY (VendorNumber) REFERENCES Vendedores(VendorNumber);
""")
conn.commit()
print("Added foreign key constraint FK_Ventas_Vendedores.")


Added foreign key constraint FK_Ventas_Productos.
Added foreign key constraint FK_Ventas_Vendedores.


TABLA PreciosCompra (purchaseprices)

In [ ]:
# Step 1: Create the "PreciosCompra" table
create_table_query = """
CREATE TABLE PreciosCompra (
    SupplierID INT, 
    Brand INT, 
    Description NVARCHAR(255), 
    Price FLOAT, 
    Classification INT, 
    PurchasePrice FLOAT, 
    VendorNumber INT, 
    VendorName NVARCHAR(255), 
    size_ml FLOAT, 
    pack_count INT
);
"""
cursor.execute(create_table_query)
conn.commit()
print("Table 'PreciosCompra' created successfully.")



In [15]:

csv_file_path = 'cleaned_purchasePrices.csv'  
purchase_prices_data = pd.read_csv(csv_file_path)

for index, row in purchase_prices_data.iterrows():
    cursor.execute("""
        INSERT INTO PreciosCompra (
            SupplierID, Brand, Description, Price, Classification, 
            PurchasePrice, VendorNumber, VendorName, size_ml, pack_count
        )
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, 
    row['SupplierID'], 
    row['Brand'], 
    row['Description'], 
    row['Price'], 
    row['Classification'], 
    row['PurchasePrice'], 
    row['VendorNumber'], 
    row['VendorName'], 
    row['size_ml'], 
    row['pack_count'])
conn.commit()
print("se insertaron los datos en 'PreciosCompra'.")




se insertaron los datos en 'PreciosCompra'.


RELACIONES

In [ ]:
# Add foreign key for Brand
cursor.execute("""
ALTER TABLE PreciosCompra
ADD CONSTRAINT FK_PreciosCompra_Productos
FOREIGN KEY (Brand) REFERENCES Productos(Brand);
""")
conn.commit()
print("Alles gut =).")

In [17]:
# Identify and resolve missing VendorNumber values
cursor.execute("""
INSERT INTO Vendedores (VendorNumber, VendorName)
SELECT DISTINCT VendorNumber, 'Unknown Vendor'
FROM PreciosCompra
WHERE VendorNumber NOT IN (SELECT VendorNumber FROM Vendedores);
""")
conn.commit()
print("Alles gut")

# Add the foreign key for VendorNumber
cursor.execute("""
ALTER TABLE PreciosCompra
ADD CONSTRAINT FK_PreciosCompra_Vendedores
FOREIGN KEY (VendorNumber) REFERENCES Vendedores(VendorNumber);
""")
conn.commit()
print("proost =).")


Alles gut
proost =).


de nuevo tengo problemas con la columna pk que hice (en este caso lo habia hecho usando otras columnas..pero me aparecen columnas nulas asique borro la columna y creo una nueva con datos artificiales nuevamente)

In [19]:
try:
    # Step 1: Drop the existing SupplierID column
    cursor.execute("""
    ALTER TABLE PreciosCompra
    DROP COLUMN SupplierID;
    """)
    conn.commit()
    print("se elimino la columna SupplierID.")

    # Step 2: Add a new SupplierID column with auto-increment
    cursor.execute("""
    ALTER TABLE PreciosCompra
    ADD SupplierID INT IDENTITY(1,1) PRIMARY KEY;
    """)
    conn.commit()
    print("se creo una nueva columna de supplierID")

except pyodbc.Error as e:
    print(f"Error: {e}")



se elimino la columna SupplierID.
se creo una nueva columna de supplierID


In [21]:

csv_file_path = 'cleaned_purchases.csv'  # Cambia esta ruta por la tuya
data = pd.read_csv(csv_file_path)

# Crear la tabla sin la columna 'Size'
create_table_query = """
CREATE TABLE Compras (
    PurchaseID NVARCHAR(50),
    InventoryId NVARCHAR(50),
    Store INT,
    Brand INT,
    Description NVARCHAR(255),
    VendorNumber INT,
    VendorName NVARCHAR(255),
    PONumber INT,
    PODate DATE,
    ReceivingDate DATE,
    InvoiceDate DATE,
    PayDate DATE,
    PurchasePrice FLOAT,
    Quantity INT,
    Dollars FLOAT,
    Classification INT,
    size_ml FLOAT,
    pack_count INT
);
"""
cursor.execute(create_table_query)
conn.commit()
print("Tabla 'Compras' creada exitosamente.")

# Cargar los datos en la tabla
for index, row in data.iterrows():
    cursor.execute("""
        INSERT INTO Compras (
            PurchaseID, InventoryId, Store, Brand, Description,
            VendorNumber, VendorName, PONumber, PODate, ReceivingDate,
            InvoiceDate, PayDate, PurchasePrice, Quantity, Dollars,
            Classification, size_ml, pack_count
        )
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, 
    row['PurchaseID'], 
    row['InventoryId'], 
    row['Store'], 
    row['Brand'], 
    row['Description'], 
    row['VendorNumber'], 
    row['VendorName'], 
    row['PONumber'], 
    row['PODate'], 
    row['ReceivingDate'], 
    row['InvoiceDate'], 
    row['PayDate'], 
    row['PurchasePrice'], 
    row['Quantity'], 
    row['Dollars'], 
    row['Classification'], 
    row['size_ml'], 
    row['pack_count'])
conn.commit()
print("Datos cargados exitosamente en la tabla 'Compras'.")

Tabla 'Compras' creada exitosamente.
Datos cargados exitosamente en la tabla 'Compras'.


In [26]:
try:
    # Establecer la relación Brand -> Productos
    cursor.execute("""
    ALTER TABLE Compras
    ADD CONSTRAINT FK_Compras_Productos
    FOREIGN KEY (Brand) REFERENCES Productos(Brand);
    """)
    conn.commit()
    print("Relación con la tabla 'Productos' establecida exitosamente.")

    # Establecer la relación VendorNumber -> Vendedores
    cursor.execute("""
    ALTER TABLE Compras
    ADD CONSTRAINT FK_Compras_Vendedores
    FOREIGN KEY (VendorNumber) REFERENCES Vendedores(VendorNumber);
    """)
    conn.commit()
    print("Relación con la tabla 'Vendedores' establecida exitosamente.")

except pyodbc.Error as e:
    print(f"Error al establecer relaciones: {e}")



Error al establecer relaciones: ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'FK_Compras_Productos' in the database. (2714) (SQLExecDirectW); [42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Could not create constraint or index. See previous errors. (1750)")


al establecer la fk con vendedores hubo un vendedornumber que no me aparecia en la tabla de vendedores. lo agregue con nombre desconocido. 

In [29]:

try:
    # Verificar valores nulos o duplicados en PurchaseID
    print("Verificando valores nulos o duplicados en 'PurchaseID'...")
    cursor.execute("""
    SELECT PurchaseID, COUNT(*)
    FROM Compras
    GROUP BY PurchaseID
    HAVING COUNT(*) > 1;
    """)
    duplicates = cursor.fetchall()
    if duplicates:
        print("Existen duplicados en 'PurchaseID':")
        for row in duplicates:
            print(row)
        raise Exception("Elimina los duplicados antes de continuar.")

    cursor.execute("""
    SELECT *
    FROM Compras
    WHERE PurchaseID IS NULL;
    """)
    nulls = cursor.fetchall()
    if nulls:
        print("Existen valores nulos en 'PurchaseID':")
        for row in nulls:
            print(row)
        raise Exception("Elimina los valores nulos antes de continuar.")

    # Establecer la clave primaria
    print("Estableciendo 'PurchaseID' como clave primaria...")
    cursor.execute("""
    ALTER TABLE Compras
    ADD CONSTRAINT PK_Compras PRIMARY KEY (PurchaseID);
    """)
    conn.commit()
    print("'PurchaseID' ahora es clave primaria en la tabla 'Compras'.")

except pyodbc.Error as e:
    print(f"Error al establecer 'PurchaseID' como clave primaria: {e}")
except Exception as custom_error:
    print(custom_error)


Verificando valores nulos o duplicados en 'PurchaseID'...
Estableciendo 'PurchaseID' como clave primaria...
Error al establecer 'PurchaseID' como clave primaria: ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot define PRIMARY KEY constraint on nullable column in table 'Compras'. (8111) (SQLExecDirectW); [42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Could not create constraint or index. See previous errors. (1750)")


cuando verifico en sql no hay ningun valor nulo, ni ningun problema visible
ALTER TABLE Compras
ALTER COLUMN PurchaseID NVARCHAR(250) NOT NULL;

In [ ]:
import pandas as pd
import pyodbc

# Ruta del archivo CSV
csv_file_path = 'cleaned_endInventory.csv'

# Leer el archivo CSV
data = pd.read_csv(csv_file_path)

#Crear la tabla 'Inventario'
#create_table_query = """
CREATE TABLE InventarioFinal (
    InventoryId NVARCHAR(250) PRIMARY KEY, -- Clave primaria
    Store INT,
    City NVARCHAR(100),
    Brand INT,
    Description NVARCHAR(255),
    onHand INT,
    Price FLOAT,
    endDate DATE,
    size_ml FLOAT,
    pack_count FLOAT
);

#"""
cursor.execute(create_table_query)
conn.commit()
print("Tabla 'Inventario Final' creada exitosamente.")



In [38]:
# Recorrer fila por fila y verificar problemas
problematic_rows = []

for index, row in data.iterrows():
    try:
        # Intentar preparar los datos para la inserción
        size_ml = float(row['size_ml']) if pd.notnull(row['size_ml']) else None
        pack_count = float(row['pack_count']) if pd.notnull(row['pack_count']) else None
        
        # Verificar si los datos son válidos para SQL Server
        cursor.execute("""
            INSERT INTO InventarioFinal (
                InventoryId, Store, City, Brand, Description, 
                onHand, Price, endDate, size_ml, pack_count
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """,
        row['InventoryId'],
        row['Store'],
        row['City'],
        row['Brand'],
        row['Description'],
        row['onHand'],
        row['Price'],
        row['endDate'],
        size_ml,
        pack_count)
    except Exception as e:
        # Agregar la fila a la lista de filas problemáticas
        problematic_rows.append((index, row.to_dict(), str(e)))

# Mostrar filas problemáticas
print(f"Se encontraron {len(problematic_rows)} filas problemáticas.")
for problem in problematic_rows:
    print(f"Fila: {problem[0]}, Error: {problem[2]}")
    print(problem[1])


Se encontraron 0 filas problemáticas.


In [40]:
# Obtener las claves existentes en la tabla 'InventarioFinal'
cursor.execute("SELECT InventoryId FROM InventarioFinal")
existing_ids = set(row[0] for row in cursor.fetchall())

# Filtrar filas que no están en la base de datos
data = data[~data['InventoryId'].isin(existing_ids)]
print(f"Filas para insertar:\n{data}")


Filas para insertar:
Empty DataFrame
Columns: [InventoryId, Store, City, Brand, Description, onHand, Price, endDate, size_ml, pack_count]
Index: []


In [41]:
try:
    # Verificar consistencia de datos
    cursor.execute("""
        SELECT DISTINCT Brand
        FROM InventarioFinal
        WHERE Brand NOT IN (SELECT Brand FROM Productos);
    """)
    missing_brands = cursor.fetchall()
    
    if missing_brands:
        print("Existen marcas en InventarioFinal que no están en Productos:")
        for brand in missing_brands:
            print(brand)
        print("Por favor, corrige estas inconsistencias antes de continuar.")
    else:
        # Establecer la clave foránea
        cursor.execute("""
            ALTER TABLE InventarioFinal
            ADD CONSTRAINT FK_InventarioFinal_Productos
            FOREIGN KEY (Brand) REFERENCES Productos (Brand);
        """)
        conn.commit()
        print("Clave foránea creada exitosamente.")

except pyodbc.Error as e:
    print(f"Error al establecer la clave foránea: {e}")



Clave foránea creada exitosamente.


TABLA FACTURAS DE COMPRAS

In [ ]:
# Crear la tabla en SQL Server 
create_table_query = """
CREATE TABLE FacturasDeCompras (
    InvoicePurchaseID INT PRIMARY KEY,
    VendorNumber INT,
    VendorName NVARCHAR(255),
    InvoiceDate DATE,
    PONumber INT,
    PODate DATE,
    PayDate DATE,
    Quantity INT,
    Dollars FLOAT,
    Freight FLOAT,
    YearMonth NVARCHAR(7),
    Interval_Days INT,
    FreightPercentage FLOAT
);
"""

cursor.execute(create_table_query)
conn.commit()
print("Tabla 'FacturasDeCompras' creada exitosamente.")


Cargar los datos

In [6]:

# Leer datos del archivo CSV
csv_file = 'InvoicePurchases.csv'  # Ruta al archivo CSV
data = pd.read_csv(csv_file)

# Insertar datos en la tabla SQL
for index, row in data.iterrows():
    insert_query = """
    INSERT INTO FacturasDeCompras (
        InvoicePurchaseID, VendorNumber, VendorName, InvoiceDate, PONumber, PODate, PayDate,
        Quantity, Dollars, Freight, YearMonth, Interval_Days, FreightPercentage
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    cursor.execute(insert_query, row['InvoicePurchaseID'], row['VendorNumber'], row['VendorName'], row['InvoiceDate'],
                   row['PONumber'], row['PODate'], row['PayDate'], row['Quantity'], row['Dollars'], row['Freight'],
                   row['YearMonth'], row['Interval_Days'], row['FreightPercentage'])

conn.commit()
print("Datos insertados exitosamente en la tabla 'FacturasDeCompras'.")




Datos insertados exitosamente en la tabla 'FacturasDeCompras'.


crear tabla INVENTARIO INICIAL

In [10]:
# Crear la tabla InventarioInicial en SQL Server
create_table_query = """
CREATE TABLE InventarioInicial (
    InventoryId NVARCHAR(50) PRIMARY KEY,
    Store INT,
    City NVARCHAR(255),
    Brand INT,
    Description NVARCHAR(255),
    onHand INT,
    Price FLOAT,
    startDate DATE,
    size_ml FLOAT,
    pack_count INT
);
"""

try:
    cursor.execute(create_table_query)
    conn.commit()
    print("Tabla 'InventarioInicial' creada exitosamente.")
except pyodbc.Error as e:
    print(f"Error al crear la tabla: {e}")


Tabla 'InventarioInicial' creada exitosamente.


In [11]:

# Cargar datos desde el archivo CSV
csv_file = 'BegInventory.csv'  # Asegúrate de que el archivo esté en el mismo directorio
data = pd.read_csv(csv_file)

# Insertar datos en la tabla SQL
insert_query = """
INSERT INTO InventarioInicial (InventoryId, Store, City, Brand, Description, onHand, Price, startDate, size_ml, pack_count)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
"""

try:
    for index, row in data.iterrows():
        cursor.execute(insert_query, row['InventoryId'], row['Store'], row['City'], row['Brand'], row['Description'],
                       row['onHand'], row['Price'], row['startDate'], row['size_ml'], row['pack_count'])
    conn.commit()
    print("Datos insertados exitosamente en la tabla 'InventarioInicial'.")
except pyodbc.Error as e:
    print(f"Error al insertar datos: {e}")



Datos insertados exitosamente en la tabla 'InventarioInicial'.
